#note: lab 2 is under lab1

# lab 1

# step 1

In [34]:
#import Necessary Libraries
import nltk
import pandas as pd
import re

!pip install camel-tools
!camel_data full
!pip install -U farasapy


from farasa.pos import FarasaPOSTagger
from farasa.ner import FarasaNamedEntityRecognizer
from farasa.diacratizer import FarasaDiacritizer
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer as stemmer

# Import the dediac_ar function from camel_tools to remove diacritics (Tashkeel)
from camel_tools.utils.dediac import dediac_ar

# Download NLTK stopwords
nltk.download('stopwords')

# Import the stopwords module
from nltk.corpus import stopwords

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.5/124.5 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# step 2

In [35]:
# Load the dataset
df = pd.read_csv('/content/ArSarcasm_train.csv')
# Inspect the dataset
print("Original Dataset:")
print(df.head()) #shows the fisrt 5 rows
print("Dataset Info:")
print(df.info()) #shows the data information

Original Dataset:
  dialect  sarcasm sentiment original_sentiment  \
0    gulf    False  negative           negative   
1     msa    False   neutral           positive   
2   egypt    False   neutral            neutral   
3  levant     True   neutral           negative   
4     msa    False   neutral           negative   

                                               tweet   source  
0  "نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ...  semeval  
1  "#نادين_نسيب_نجيم ❤️❤️❤️مجلة #ماري_كلير 💭#ملكة...  semeval  
2                      "@Alito_NBA اتوقع انه بيستمر"  semeval  
3     "@KSA24 يعني "بموافقتنا" لأن دمشق صايرة موسكو"  semeval  
4  "RT @alaahmad20: قائد في الحرس يعترف بفقدان ال...  semeval  
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8437 entries, 0 to 8436
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             8437 non-null   object
 1   sarcasm             84

# step 3

In [36]:

def normalize_text(text):
    # Replace underscore with a space
    text = text.replace('_', ' ')
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove Tatweel
    text = text.replace('ـ', '')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

#  Apply normalization to the 'tweet' column
df['tweet'] = df['tweet'].apply(normalize_text)
print("After Normalization:")
print(df['tweet'].head())

After Normalization:
0    نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ك...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد في الحرس يعترف بفقدان السيطرة الأمنية في ...
Name: tweet, dtype: object


# step 4

In [37]:

# Define a function to remove diacritics from Arabic text
def remove_diacritics(text):
 # Use dediac_ar to strip diacritics from the input text
    return dediac_ar(text)

# Apply the diacritic removal function to the 'tweet' column in the dataset
df['tweet'] = df['tweet'].apply(remove_diacritics)

# Print the first few rows of the 'tweet' column after removing diacritics
print("After Removing Diacritics:")
print(df['tweet'].head())

After Removing Diacritics:
0    نصيحه ما عمرك اتنزل لعبة سوبر ماريو مش زي ما ك...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد في الحرس يعترف بفقدان السيطرة الأمنية في ...
Name: tweet, dtype: object


# step 6

In [38]:


# Load Arabic stopwords from NLTK
arabic_stopwords = set(stopwords.words('arabic'))

# Define the stopword removal function
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word not in arabic_stopwords]  # Filter stopwords
    return ' '.join(filtered_words)  # Rejoin the filtered words into a string

# Apply the function to the 'tweet' column
df['tweet'] = df['tweet'].apply(remove_stopwords)

# Display the cleaned text
print("After Removing Stopwords:")
print(df['tweet'].head())

After Removing Stopwords:
0    نصيحه عمرك اتنزل لعبة سوبر ماريو مش زي كنا متو...
1          نادين نسيب نجيم مجلة ماري كلير ملكة الصحراء
2                                     اتوقع انه بيستمر
3                  يعني بموافقتنا لأن دمشق صايرة موسكو
4    قائد الحرس يعترف بفقدان السيطرة الأمنية شرقي و...
Name: tweet, dtype: object


# test dataset

# step 2

In [39]:
# Load the dataset

df1 = pd.read_csv('/content/ArSarcasm_test.csv')


In [40]:
# Inspect the dataset
print("Original Dataset:")
print(df1.head()) #shows the fisrt 5 rows
print("Dataset Info:")
print(df1.info()) #shows the data information

Original Dataset:
  dialect  sarcasm sentiment original_sentiment  \
0     msa     True  negative           negative   
1    gulf    False  positive            neutral   
2     msa     True   neutral            neutral   
3     msa    False   neutral            neutral   
4     msa    False   neutral            neutral   

                                               tweet   source  
0  "@AbuEmad74241481 @Cesars2014 هههههه حزب الله ...  semeval  
1  "RT @JannetForster: البنات اللي م صامو بقولكم ...  semeval  
2             اشارة رابعة اشبه بنار تحرق الانقلابيين     astd  
3  "@EGYPTAIR ماهي مميزات درجه بزنس علماً اني في ...  semeval  
4  ما لا تراه على التلفاز منافسة شديدة بين المرشح...  semeval  
Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2110 entries, 0 to 2109
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   dialect             2110 non-null   object
 1   sarcasm             21

# step 3

In [41]:

def normalize_text(text):
    # Replace underscore with a space
    text = text.replace('_', ' ')
    # Remove non-Arabic characters
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    # Remove Tatweel
    text = text.replace('ـ', '')
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply normalization to the 'tweet' column
df1['tweet'] = df1['tweet'].apply(normalize_text)
print("After Normalization:")
print(df1['tweet'].head())

After Normalization:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي م صامو بقولكم ترا رمضان قرب و الوق...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علماً اني في بروكسيل و ا...
4    ما لا تراه على التلفاز منافسة شديدة بين المرشح...
Name: tweet, dtype: object


# step 4

In [42]:

# Define a function to remove diacritics from Arabic text
def remove_diacritics(text):
 # Use dediac_ar to strip diacritics from the input text
    return dediac_ar(text)

# Apply the diacritic removal function to the 'tweet' column in the dataset
df1['tweet'] = df1['tweet'].apply(remove_diacritics)

# Print the first few rows of the 'tweet' column after removing diacritics
print("After Removing Diacritics:")
print(df1['tweet'].head())

After Removing Diacritics:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي م صامو بقولكم ترا رمضان قرب و الوق...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علما اني في بروكسيل و ار...
4    ما لا تراه على التلفاز منافسة شديدة بين المرشح...
Name: tweet, dtype: object


# step 6

In [43]:

# Load Arabic stopwords from NLTK
arabic_stopwords = set(stopwords.words('arabic'))

# Define the stopword removal function
def remove_stopwords(text):
    words = text.split()  # Split the text into words
    filtered_words = [word for word in words if word not in arabic_stopwords]  # Filter stopwords
    return ' '.join(filtered_words)  # Rejoin the filtered words into a string

# Apply the function to the 'tweet' column
df1['tweet'] = df1['tweet'].apply(remove_stopwords)

# Display the cleaned text
print("After Removing Stopwords:")
print(df1['tweet'].head())

After Removing Stopwords:
0    هههههه حزب الله حطموا اسطورة الميركافا الاسرائ...
1    البنات اللي صامو بقولكم ترا رمضان قرب الوقت قل...
2               اشارة رابعة اشبه بنار تحرق الانقلابيين
3    ماهي مميزات درجه بزنس علما اني بروكسيل اريد ال...
4    تراه التلفاز منافسة شديدة المرشحين هيلاري كلنت...
Name: tweet, dtype: object


# lab2

#step 1

In [44]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report



In [45]:
# Load the datasets
df = pd.read_csv('/content/ArSarcasm_train.csv') #train set
df1 = pd.read_csv('/content/ArSarcasm_test.csv') #test set


In [46]:

# Display column names for train dataset
print("Column names:", df.columns.tolist())
# Display column names for test dataset
print("Column names:", df1.columns.tolist())


Column names: ['dialect', 'sarcasm', 'sentiment', 'original_sentiment', 'tweet', 'source']
Column names: ['dialect', 'sarcasm', 'sentiment', 'original_sentiment', 'tweet', 'source']


In [47]:

# Manually split the data into training and testing sets
X_train = df['tweet']
X_test = df1['tweet']
y_train = df['sentiment']
y_test = df1['sentiment']


In [48]:

# Print dataset shapes
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)


Shape of X_train: (8437,)
Shape of X_test: (2110,)
Shape of y_train: (8437,)
Shape of y_test: (2110,)


#step 2

In [49]:
 #Step 2: Feature Extraction using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2), max_features=10000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# step 3

In [50]:
#Train and Evaluate Models using Random Forest
model = RandomForestClassifier(n_estimators=300, max_depth=50, random_state=42)
model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Calculate model accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Model accuracy:", accuracy)

# Generate a detailed classification report
report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


Model accuracy: 0.6123222748815166
Classification Report:
               precision    recall  f1-score   support

    negative       0.56      0.58      0.57       716
     neutral       0.64      0.80      0.71      1078
    positive       0.74      0.04      0.08       316

    accuracy                           0.61      2110
   macro avg       0.64      0.47      0.45      2110
weighted avg       0.63      0.61      0.57      2110



seeing if there imbalence

In [51]:
print(df['sentiment'].value_counts())


sentiment
neutral     4262
negative    2813
positive    1362
Name: count, dtype: int64
